In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import psycopg2
from sqlalchemy import create_engine, MetaData, Table, Column, Integer, String, Float
from api_keys import client_id
from twitch import TwitchClient
from pprint import pprint

In [57]:
csvpath = './Priya_Notebooks/Website/static/csv/'

,channel_created_at,channel_followers,channel_game,channel_lan,channel_mature,channel_partner,channel_views,channel_description,stream_viewers
0,2014-09-12 23:50:05.989719,5512812,Minecraft,en,False,True,327236000,Overwatch Professional tank player and full ti...,108196
1,2014-03-20 22:28:22.813998,2838193,Just Chatting,en,False,True,14163987,Youtube.com/AdinRoss ...,83324
2,2011-11-15 01:29:29.140794,3254322,Grand Theft Auto V,en,False,True,53113707,Hi !,71644
3,2018-06-08 21:13:24.389039,716366,Dungeons & Dragons,en,False,True,22412671,Welcome to Critical Role! This is the live hom...,54438
4,2013-11-14 00:27:41.456706,4231262,League of Legends,en,False,True,233324328,Season 6 Challenger | Most REFORMED Player NA ...,42102


In [2]:
client = TwitchClient(client_id= f'{client_id}')

In [52]:
#getting live streams data

live_streams = client.streams.get_live_streams(limit = 100)
pprint(live_streams[0])

#lsdf = pd.DataFrame.from_dict(live_streams[0].channel, orient = 'index')










{'average_fps': 60,
 'broadcast_platform': 'live',
 'channel': {'broadcaster_language': 'en',
             'broadcaster_software': '',
             'broadcaster_type': '',
             'created_at': datetime.datetime(2014, 3, 20, 22, 28, 22, 813998),
             'description': 'Youtube.com/AdinRoss                                                        '
                            'Twitter.com/adinross',
             'display_name': 'AdinRoss',
             'followers': 2854562,
             'game': 'Just Chatting',
             'id': 59299632,
             'language': 'en',
             'logo': 'https://static-cdn.jtvnw.net/jtv_user_pictures/10c197cb-295f-4d40-8a6c-26a0a5da22ee-profile_image-300x300.png',
             'mature': False,
             'name': 'adinross',
             'partner': True,
             'privacy_options_enabled': False,
             'private_video': False,
             'profile_banner': 'https://static-cdn.jtvnw.net/jtv_user_pictures/ab24c36c-51f8-4872-a8db-33

In [32]:
lsdf = pd.DataFrame.from_dict(live_streams[0].channel, orient = 'index')
#live_streams[0].values()
lsdf.transpose()

,mature,status,broadcaster_language,broadcaster_software,display_name,game,language,id,name,created_at,...,video_banner,profile_banner,profile_banner_background_color,url,views,followers,broadcaster_type,description,private_video,privacy_options_enabled
0,False,EXCEPTIONALLY UNLUCKY GAMING WIZARD SMASHES AL...,en,,xQcOW,Minecraft,en,71092938,xqcow,2014-09-12 23:50:05.989719,...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,,https://www.twitch.tv/xqcow,327236000,5512812,,Overwatch Professional tank player and full ti...,False,False


In [62]:
channels = []
game_name = []
viewers = []

channel_created_at = []
channel_followers = []
channel_id = []
channel_display_name  = []
channel_game = []
channel_lan = []
channel_mature = []
channel_partner = []
channel_views = []
channel_description = []
for game in live_streams: 
    channel_created_at.append(game.channel.created_at)
    channel_followers.append(game.channel.followers)
    channel_game.append(game.channel.game)
    channel_lan.append(game.channel.language)
    channel_mature.append(game.channel.mature)
    channel_partner.append(game.channel.partner)
    channel_views.append(game.channel.views)
    channel_description.append(game.channel.description)
    channel_id.append(game.channel.id)
    channel_display_name.append(game.channel.display_name)
    viewers.append(game.viewers)
    
toplivestreams = pd.DataFrame({
    "channel_id":channel_id,
    "channel_display_name":channel_display_name,
    "channel_description" : channel_description,
    "channel_created_at" : channel_created_at,
    "channel_followers" : channel_followers,
    "channel_game" : channel_game,
    "channel_lan" : channel_lan,
    "channel_mature" : channel_mature,
    "channel_partner" : channel_partner,
    "channel_views" : channel_views,
    "stream_viewers" : viewers})

toplivestreams.head(5+1)


,channel_id,channel_display_name,channel_description,channel_created_at,channel_followers,channel_game,channel_lan,channel_mature,channel_partner,channel_views,stream_viewers
0,59299632,AdinRoss,Youtube.com/AdinRoss ...,2014-03-20 22:28:22.813998,2854562,Just Chatting,en,False,True,14257157,130093
1,569325723,loud_coringa,"Eaeeee! Meu nome é Victor, mais conhecido como...",2020-08-19 03:18:41.005557,1688782,Grand Theft Auto V,pt-br,False,True,16404958,101408
2,71092938,xQcOW,Overwatch Professional tank player and full ti...,2014-09-12 23:50:05.989719,5516226,DEVOUR,en,False,True,328189984,95340
3,26154978,Sykkuno,Hi !,2011-11-15 01:29:29.140794,3258948,Grand Theft Auto V,en,False,True,53333877,66902
4,638065882,karlnetwork,Shut up dont tell anyone stop stop stop,2021-01-18 09:27:29.883783,362185,Golf With Your Friends,en,False,False,551392,57725
5,51496027,loltyler1,Season 6 Challenger | Most REFORMED Player NA ...,2013-11-14 00:27:41.456706,4231883,League of Legends,en,False,True,233648974,41450


In [64]:
toplivestreams.to_csv(csvpath+'toplivestreams.csv', index = False, header = True)

In [30]:
df = pd.Panel(live_streams[0])

AttributeError: module 'pandas' has no attribute 'Panel'

,vid_title,vid_length,video_views,total_view_time-calc-hours,channel_id,channel_display_name,channel_description,channel_created_at,channel_followers,channel_game,channel_lan,channel_mature,channel_partner,channel_views
0,MARBELLA VICE #4 || Vamos a conseguir dinerito 🥵,18172,4612938,23285085.93,59299632,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
1,MARBELLA VICE #5 || En busca de un gran botín 🤡,15220,3926790,16601595.50,569325723,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
2,MARBELLA VICE #7 || día de buscar empleados,17252,4245320,20344516.84,71092938,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
3,MARBELLA VICE #6 || Pirulas,11331,3170583,9979409.99,26154978,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
4,DIAS TEMPOS RUINS FAZEM AS PESSOAS MAIS FORTES...,33566,2772568,25851115.97,638065882,loud_coringa,"Eaeeee! Meu nome é Victor, mais conhecido como...",2020-08-19 03:18:41.005557,1689843,Grand Theft Auto V,pt-br,False,True,16442806
5,MARBELLA VICE #8 || Empezamos a tener mucho ma...,13855,2326087,8952204.27,51496027,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055


In [65]:
top_videos = client.videos.get_top(limit = 100)
pprint(top_videos[1])

{'animated_preview_url': 'https://dgeft87wbj63p.cloudfront.net/46989779e4766b5fef0b_auronplay_41807275580_1618754838/storyboards/992088534-strip-0.jpg',
 'broadcast_id': 41807275580,
 'broadcast_type': 'archive',
 'channel': {'broadcaster_language': 'es',
             'broadcaster_software': 'unknown_rtmp',
             'broadcaster_type': 'partner',
             'created_at': datetime.datetime(2019, 9, 3, 14, 2, 49, 15250),
             'description': 'feliz pero no mucho ',
             'display_name': 'auronplay',
             'followers': 8236862,
             'game': 'Grand Theft Auto V',
             'id': 459331509,
             'language': 'es',
             'logo': 'https://static-cdn.jtvnw.net/jtv_user_pictures/ec898e4a-e0df-4dc0-a99d-7540c6dbe1e8-profile_image-300x300.png',
             'mature': True,
             'name': 'auronplay',
             'partner': True,
             'privacy_options_enabled': False,
             'private_video': False,
             'profile_banne

In [74]:
channels1 = []
game_name1 = []
views1 = []
vid_length = []
vid_title = []
vid_total_views = []

channel_created_at1 = []
channel_followers1 = []
channel_id1 = []
channel_display_name1  = []
channel_game1 = []
channel_lan1 = []
channel_mature1 = []
channel_partner1 = []
channel_views1 = []
channel_description1 = []
for game in top_videos: 
    channel_created_at1.append(game.channel.created_at)
    channel_followers1.append(game.channel.followers)
    channel_game1.append(game.channel.game)
    channel_lan1.append(game.channel.language)
    channel_mature1.append(game.channel.mature)
    channel_partner1.append(game.channel.partner)
    channel_views1.append(game.channel.views)
    channel_description1.append(game.channel.description)
    channel_id1.append(game.channel.id)
    channel_display_name1.append(game.channel.display_name)
    views1.append(game.views)
    vid_length.append(game.length)
    vid_title.append(game.title)
    vid_total_views.append(round(((game.views*game.length)/(60*60)),2))
    
topvideos = pd.DataFrame({
    "vid_title":vid_title,
    "vid_length":vid_length,
    "video_views" : views1,
    "total_view_time-calc-hours":vid_total_views,
    "channel_id":channel_id,
    "channel_display_name":channel_display_name1,
    "channel_description" : channel_description1,
    "channel_created_at" : channel_created_at1,
    "channel_followers" : channel_followers1,
    "channel_game" : channel_game1,
    "channel_lan" : channel_lan1,
    "channel_mature" : channel_mature1,
    "channel_partner" : channel_partner1,
    "channel_views" : channel_views1,
    
    
    })

topvideos.head(5+1)

,vid_title,vid_length,video_views,total_view_time-calc-hours,channel_id,channel_display_name,channel_description,channel_created_at,channel_followers,channel_game,channel_lan,channel_mature,channel_partner,channel_views
0,MARBELLA VICE #4 || Vamos a conseguir dinerito 🥵,18172,4612938,23285085.93,59299632,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
1,MARBELLA VICE #5 || En busca de un gran botín 🤡,15220,3926790,16601595.50,569325723,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
2,MARBELLA VICE #7 || día de buscar empleados,17252,4245320,20344516.84,71092938,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
3,MARBELLA VICE #6 || Pirulas,11331,3170583,9979409.99,26154978,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055
4,DIAS TEMPOS RUINS FAZEM AS PESSOAS MAIS FORTES...,33566,2772568,25851115.97,638065882,loud_coringa,"Eaeeee! Meu nome é Victor, mais conhecido como...",2020-08-19 03:18:41.005557,1689843,Grand Theft Auto V,pt-br,False,True,16442806
5,MARBELLA VICE #8 || Empezamos a tener mucho ma...,13855,2326087,8952204.27,51496027,auronplay,feliz pero no mucho,2019-09-03 14:02:49.015250,8236862,Grand Theft Auto V,es,True,True,126233055


In [76]:
topvideos.to_csv(csvpath+'topvideos.csv', index = False, header = True)

In [84]:
toplivestreamgames = toplivestreams.channel_game.unique()
toplivestreamgames[0]

'Just Chatting'

In [86]:
gamesummary = client.stream.get_summary(toplivestreamgames[0])

AttributeError: 'TwitchClient' object has no attribute 'stream'

In [90]:
topvidchan = topvideos.channel_display_name.unique()

In [91]:
topstreamchan = toplivestreams.channel_display_name.unique()

In [93]:
topchan = set(topvidchan).intersection(topstreamchan)
topchan

{'Ninja', 'Sykkuno', 'juansguarnizo', 'loud_coringa', 'xQcOW'}

In [79]:
serverlocations = []
for server in servers:
    serverlocations.append(server.name)
serverlocations

['Europe: Germany, Frankfurt (2)',
 'NA: Mexico, Queretaro (1)',
 'Asia: Japan, Tokyo (1)',
 'Asia: India, Chennai',
 'US Central: Dallas, TX',
 'Asia: Thailand, Bangkok',
 'US East: Chicago, IL (2)',
 'Asia: Singapore',
 'Europe: Finland, Helsinki',
 'Europe: Norway, Oslo',
 'Australia: Sydney (2)',
 'US West: Salt Lake City, UT',
 'Europe: Sweden, Stockholm',
 'US West: Seattle, WA',
 'NA: Canada, Quebec',
 'Asia: Taiwan, Taipei (1)',
 'US East: Miami, FL',
 'South America: Brazil, Sao Paulo',
 'Europe: Germany, Düsseldorf (1)',
 'Asia: Japan, Tokyo (3)',
 'Europe: UK, London (3)',
 'US West: Los Angeles, CA',
 'NA: Mexico, Queretaro (2)',
 'Asia: Hong Kong',
 'Europe: France, Marseille',
 'US East: Ashburn, VA (3)',
 'US East: Ashburn, VA (5)',
 'US West: Phoenix, AZ',
 'Europe: Netherlands, Amsterdam (3)',
 'US East: Atlanta, GA',
 'Asia: South Korea, Seoul (3)',
 'US West: San Francisco, CA',
 'Europe: Czech Republic, Prague',
 'Australia: Sydney (1)',
 'US East: New York, NY',
 '

In [35]:
servers = client.ingests.get_server_list()
pprint(servers)

[{'availability': 1.0,
  'default': False,
  'id': 1,
  'name': 'Europe: Germany, Frankfurt (2)',
  'url_template': 'rtmp://fra02.contribute.live-video.net/app/{stream_key}'},
 {'availability': 1.0,
  'default': False,
  'id': 2,
  'name': 'NA: Mexico, Queretaro (1)',
  'url_template': 'rtmp://qro01.contribute.live-video.net/app/{stream_key}'},
 {'availability': 1.0,
  'default': False,
  'id': 3,
  'name': 'Asia: Japan, Tokyo (1)',
  'url_template': 'rtmp://tyo01.contribute.live-video.net/app/{stream_key}'},
 {'availability': 1.0,
  'default': False,
  'id': 4,
  'name': 'Asia: India, Chennai',
  'url_template': 'rtmp://maa01.contribute.live-video.net/app/{stream_key}'},
 {'availability': 1.0,
  'default': False,
  'id': 5,
  'name': 'US Central: Dallas, TX',
  'url_template': 'rtmp://dfw.contribute.live-video.net/app/{stream_key}'},
 {'availability': 1.0,
  'default': False,
  'id': 6,
  'name': 'Asia: Thailand, Bangkok',
  'url_template': 'rtmp://bkk.contribute.live-video.net/app/{s